In [3]:
import pandas as pd
import numpy as np
import subprocess
import seaborn as sns
import io
import os
import pandas as pd
import glob

def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

In [4]:
metadata = pd.read_csv("../hifi_metadata.csv")

In [23]:
for i in metadata["Baums_db_id"].tolist():
    task = "longphase_modcall"
    mem = "128"
    cpus = "20"
    inpath = '/storage/home/nsl5160/scratch/darpa/mapped'
    outpath = '../calls'
    prefix = str(i)
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         inpath + " " + outpath + " " + prefix], shell=True)

Submitted batch job 5178029
Submitted batch job 5178030
Submitted batch job 5178031
Submitted batch job 5178032
Submitted batch job 5178033
Submitted batch job 5178034


In [5]:
alphabet = [chr(x) for x in range(97, 123)]
chunks = [letter1+letter2 for letter1 in alphabet for letter2 in alphabet][0:78]

In [8]:
for i in chunks:
    task = "bcftools_call_var_ccs"
    mem = "32"
    cpus = "4"
    chunk = i
    bampath = '../apal_imputation_bam_hifi.list'
    outpath = "/storage/group/ibb3/default/darpa/calls/"
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         outpath + " " + bampath + " " + chunk], shell=True)

Submitted batch job 5273824
Submitted batch job 5273825
Submitted batch job 5273826
Submitted batch job 5273827
Submitted batch job 5273828
Submitted batch job 5273829
Submitted batch job 5273830
Submitted batch job 5273831
Submitted batch job 5273832
Submitted batch job 5273833
Submitted batch job 5273834
Submitted batch job 5273835
Submitted batch job 5273836
Submitted batch job 5273837
Submitted batch job 5273838
Submitted batch job 5273839
Submitted batch job 5273840
Submitted batch job 5273841
Submitted batch job 5273842
Submitted batch job 5273843
Submitted batch job 5273844
Submitted batch job 5273845
Submitted batch job 5273846
Submitted batch job 5273847
Submitted batch job 5273848
Submitted batch job 5273849
Submitted batch job 5273850
Submitted batch job 5273851
Submitted batch job 5273852
Submitted batch job 5273853
Submitted batch job 5273854
Submitted batch job 5273855
Submitted batch job 5273856
Submitted batch job 5273857
Submitted batch job 5273858
Submitted batch job 

In [9]:
%%bash
source ~/.bashrc
conda activate ipyrad
bcftools concat ../calls/apal_imputation_panel_hifi_part_??.vcf \
    -O z > ../calls/apal_imputation_panel_hifi.vcf.gz

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


In [37]:
%%bash
for i in 20148 20151 20153 20154 20155 20156; do
    echo -e "/storage/home/nsl5160/scratch/darpa/mapped/${i}.bam\t${i}" >> ../hifi_header.txt
done

In [10]:
%%bash
source ~/.bashrc
conda activate ipyrad
bcftools reheader --threads 8 --samples ../hifi_header.txt \
    ../calls/apal_imputation_panel_hifi.vcf.gz \
    > ../calls/apal_imputation_panel_hifi_reheader.vcf.gz

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


In [11]:
%%bash
source ~/.bashrc
conda activate ipyrad
bcftools index ../calls/apal_imputation_panel_hifi_reheader.vcf.gz

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


In [45]:
%%bash
source ~/.bashrc
conda activate ipyrad
for i in 20148 20151 20153 20154 20155 20156; do
    vcftools --gzvcf ../calls/apal_imputation_panel_hifi_reheader.vcf.gz \
        --minGQ 20 --minQ 20 --recode-INFO-all \
        --indv ${i} \
        --recode --out ../calls/${i}
done

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details



VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf ../calls/apal_imputation_panel_hifi_reheader.vcf.gz
	--recode-INFO-all
	--minGQ 20
	--minQ 20
	--out ../calls/20148
	--recode
	--indv 20148

Using zlib version: 1.2.11
Keeping individuals in 'keep' list
After filtering, kept 1 out of 6 Individuals
Outputting VCF file...
After filtering, kept 4531120 out of a possible 4784593 Sites
Run Time = 61.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf ../calls/apal_imputation_panel_hifi_reheader.vcf.gz
	--recode-INFO-all
	--minGQ 20
	--minQ 20
	--out ../calls/20151
	--recode
	--indv 20151

Using zlib version: 1.2.11
Keeping individuals in 'keep' list
After filtering, kept 1 out of 6 Individuals
Outputting VCF file...
After filtering, kept 4531120 out of a possible 4784593 Sites
Run Time = 61.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:

In [46]:
for i in metadata["Baums_db_id"].tolist():
    task = "longphase_phase"
    mem = "128"
    cpus = "20"
    bampath = '/storage/home/nsl5160/scratch/darpa/mapped'
    inpath = '../calls'
    prefix = str(i)
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         bampath + " " + inpath + " " + prefix], shell=True)

Submitted batch job 5182946
Submitted batch job 5182947
Submitted batch job 5182948
Submitted batch job 5182949
Submitted batch job 5182950
Submitted batch job 5182951


In [47]:
%%bash
source ~/.bashrc
conda activate ipyrad
for i in 20148 20151 20153 20154 20155 20156; do
    bgzip ../calls/${i}_phased.vcf
    tabix ../calls/${i}_phased.vcf.gz
done

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


Lastly, we need to call CNVs so they can be removed from the panel. Could mess up imputation and phasing if we don't remove them:

In [55]:
%%bash
cut -f1-3 \
    /storage/group/ibb3/default/AP_AC_genome_seqs/dovetail_Apalm/HiC_improvement/Apalm_assembly_v3.0_200415.repeats.bed \
    > ../apal_rust_compatible_repeats.bed
sed -i "s/^/Apal_/g" ../apal_rust_compatible_repeats.bed
grep "^sym" ../references/apal_and_sym.bed >> ../apal_rust_compatible_repeats.bed

In [5]:
for i in metadata["Baums_db_id"].tolist():
    task = "hificnv"
    mem = "128"
    cpus = "20"
    bampath = '/storage/home/nsl5160/scratch/darpa/mapped'
    outpath = '../calls' 
    prefix = str(i) 
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         bampath + " " + outpath + " " + prefix], shell=True)

Submitted batch job 5218769
Submitted batch job 5218770
Submitted batch job 5218771
Submitted batch job 5218772
Submitted batch job 5218773
Submitted batch job 5218774


In [6]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd ../references/
bioawk -c fastx '{print $name"\t0\t"length($seq)}' apal2_and_sym.fa > apal2_and_sym.bed

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


In [7]:
%%bash
grep "^sym" ../references/apal2_and_sym.bed >> ../apal2_exclude.bed

In [8]:
for i in metadata["Baums_db_id"].tolist():
    task = "hificnv_2"
    mem = "128"
    cpus = "20"
    bampath = '/storage/home/nsl5160/scratch/darpa/mapped2'
    outpath = '../calls/test' 
    prefix = str(i)
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         bampath + " " + outpath + " " + prefix], shell=True)

Submitted batch job 5218897
Submitted batch job 5218898
Submitted batch job 5218899
Submitted batch job 5218900
Submitted batch job 5218901
Submitted batch job 5218902


In [90]:
for i in metadata["Baums_db_id"].tolist():
    path = glob.glob("../calls/" + str(i) + "_hificnv.*.vcf.gz")
    vcf = pd.read_csv(path[0], 
            compression = "gzip", comment = "#", header = None, sep = "\t")
    vcf = vcf[vcf[5] > 20].reset_index(drop=True)
    vcf["SVLEN"] = vcf[7].str.split(";", n = 4, expand = True)[3].str.replace("SVLEN=", "").astype(int)
    vcf["END"] = vcf[1] + vcf["SVLEN"]
    vcf = vcf[[0, 1, "END"]]
    vcf[1] = vcf[1] - 1
    vcf["END"] = vcf["END"] - 1
    vcf.set_index(0).to_csv("../calls/" + str(i) + "_hificnv.bed", sep = "\t", header = None)

In [111]:
for i in metadata["Baums_db_id"].tolist():
    subprocess.run(["cat ../calls/" + str(i) 
          + "_hificnv.bed >> ../references/repeats_and_cnv.bed"], shell=True)
for i in ["repeats"]:
    subprocess.run(["cut -f1-3 ../references/" + str(i) 
          + ".bed >> ../references/repeats_and_cnv.bed"], shell=True)
subprocess.run(["sort -k1,1 -k2,2n ../references/repeats_and_cnv.bed > ../references/repeats_and_cnv_sorted.bed"], shell=True)

CompletedProcess(args=['sort -k1,1 -k2,2n ../references/repeats_and_cnv.bed > ../references/repeats_and_cnv_sorted.bed'], returncode=0)

In [113]:
%%bash
source ~/.bashrc
conda activate ipyrad
bedtools merge -i ../references/repeats_and_cnv_sorted.bed \
    > ../references/repeats_and_cnv_sorted_merged.bed

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details
